In [3]:
!pip install python-pptx

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.8/472.8 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.3/172.3 kB 13.0 MB/s eta 0:00:00


In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os, io
from datetime import datetime
from collections import OrderedDict

def clean_col(c):
    return (
        str(c)
        .strip()
        .lower()
        .replace(" ", "_")
        .replace("-", "_")
        .replace("/", "_")
        .replace(".", "_")
    )

def first_present(keys, columns):
    for k in keys:
        if k in columns:
            return k
    return None

def load_data(path):
    for enc in ["utf-8-sig", "cp1252", "latin1"]:
        try:
            df = pd.read_csv(path, encoding=enc, engine="python", on_bad_lines="skip")
            return df, enc
        except Exception:
            continue
    raise RuntimeError("Failed to read CSV with common encodings.")

def main(csv_path, out_dir="analysis_outputs"):
    os.makedirs(out_dir, exist_ok=True)

    df, enc = load_data(csv_path)
    original_columns = df.columns.tolist()
    df.columns = [clean_col(c) for c in df.columns]

    # Parse date columns
    date_cols_guess = [c for c in df.columns if "date" in c]
    for c in date_cols_guess:
        try:
            df[c] = pd.to_datetime(df[c], errors="coerce", infer_datetime_format=True, dayfirst=False)
        except Exception:
            pass

    # Normalize money & quantities
    money_like = [c for c in df.columns if any(k in c for k in ["amount","price","revenue","sales","total"])]
    for c in money_like:
        if df[c].dtype == object:
            df[c] = (
                df[c].astype(str)
                .str.replace(",", "", regex=False)
                .str.replace("₹", "", regex=False)
                .str.replace("$", "", regex=False)
                .str.replace(" ", "", regex=False)
                .str.replace("INR", "", regex=False)
            )
            df[c] = pd.to_numeric(df[c], errors="coerce")

    qty_cols = [c for c in df.columns if c in ["qty","quantity","order_quantity","units","unit"] or "qty" in c]
    for c in qty_cols:
        df[c] = pd.to_numeric(df[c], errors="coerce")

    # Column schema mapping
    colmap = {
        "order_id": [ "order_id","orderid","order_item_id","order_item","order_item_identifier"],
        "order_date": ["order_date","date","purchase_date","order_date_time"],
        "status": ["status","order_status"],
        "fulfilment": ["fulfilment","fulfillment","fulfillment_channel","fulfilled_by"],
        "sales_channel": ["sales_channel","saleschannel","channel"],
        "category": ["category","product_category","product_type"],
        "size": ["size","variant","product_size"],
        "qty": ["qty","quantity","units","order_quantity"],
        "amount": ["amount","item_total","total","grand_total","net_amount","order_amount","sales"],
        "ship_city": ["ship_city","ship_city_name","city"],
        "ship_state": ["ship_state","state"],
        "ship_postal_code": ["ship_postal_code","postal_code","pincode","zip"],
        "courier_status": ["courier_status","shipment_status","shipping_status"],
        "fulfilled_on": ["fulfilled_on","dispatch_date","ship_date"],
    }
    schema = {k:first_present(v, df.columns) for k,v in colmap.items()}

    ORDER_ID = schema["order_id"]
    ORDER_DATE = schema["order_date"]
    STATUS = schema["status"]
    FULFIL = schema["fulfilment"]
    CHANNEL = schema["sales_channel"]
    CAT = schema["category"]
    SIZE = schema["size"]
    QTY = schema["qty"]
    AMT = schema["amount"]
    CITY = schema["ship_city"]
    STATE = schema["ship_state"]

    # KPIs
    kpi = OrderedDict()
    kpi["used_encoding"] = enc
    kpi["total_rows"] = len(df)
    kpi["columns_detected"] = ", ".join(df.columns)
    kpi["total_revenue"] = float(np.nansum(df[AMT])) if AMT else np.nan
    kpi["avg_order_value"] = float(np.nanmean(df[AMT])) if AMT else np.nan
    kpi["total_units"] = int(np.nansum(df[QTY])) if QTY else np.nan
    kpi["total_orders"] = df[ORDER_ID].nunique() if ORDER_ID else np.nan

    # Time series
    timeseries = None
    if ORDER_DATE:
        ts = df.dropna(subset=[ORDER_DATE]).copy()
        ts["year_month"] = ts[ORDER_DATE].dt.to_period("M").dt.to_timestamp()
        agg_map = {}
        if AMT: agg_map[AMT] = "sum"
        if QTY: agg_map[QTY] = "sum"
        if ORDER_ID: agg_map[ORDER_ID] = pd.Series.nunique
        if agg_map:
            timeseries = ts.groupby(["year_month"]).agg(agg_map).reset_index().sort_values("year_month")

    # Category analysis
    top_categories_sales = (df.groupby(CAT)[AMT].sum().sort_values(ascending=False).head(10).reset_index()) if CAT and AMT else None
    top_categories_units = (df.groupby(CAT)[QTY].sum().sort_values(ascending=False).head(10).reset_index()) if CAT and QTY else None

    # Size analysis
    size_summary = None
    if SIZE:
        agg_map = {}
        if AMT: agg_map[AMT] = "sum"
        if QTY: agg_map[QTY] = "sum"
        if agg_map:
            size_summary = df.groupby(SIZE).agg(agg_map).sort_values(list(agg_map.keys())[0], ascending=False).reset_index()

    # Fulfilment & Status
    fulfil_summary = None
    if FULFIL:
        fulfil_summary = df.groupby(FULFIL).size().reset_index(name="orders")
        if AMT: fulfil_summary = fulfil_summary.merge(df.groupby(FULFIL)[AMT].sum().reset_index(name="sales_amount"), on=FULFIL, how="left")
        if QTY: fulfil_summary = fulfil_summary.merge(df.groupby(FULFIL)[QTY].sum().reset_index(name="units"), on=FULFIL, how="left")

    status_summary = None
    if STATUS:
        status_summary = df.groupby(STATUS).size().reset_index(name="orders")
        if AMT: status_summary = status_summary.merge(df.groupby(STATUS)[AMT].sum().reset_index(name="sales_amount"), on=STATUS, how="left")

    # Geography
    state_summary = (df.groupby(STATE)[AMT].sum().sort_values(ascending=False).reset_index()) if STATE and AMT else None
    city_summary = (df.groupby(CITY)[AMT].sum().sort_values(ascending=False).reset_index()) if CITY and AMT else None

    # Save CSVs
    def save_df(d, name):
        if d is None: return None
        path = os.path.join(out_dir, f"{name}.csv")
        d.to_csv(path, index=False)
        return path

    outputs = {
        "timeseries": save_df(timeseries, "timeseries_monthly"),
        "top_categories_sales": save_df(top_categories_sales, "top_categories_by_sales"),
        "top_categories_units": save_df(top_categories_units, "top_categories_by_units"),
        "size_summary": save_df(size_summary, "size_summary"),
        "fulfilment_summary": save_df(fulfil_summary, "fulfilment_summary"),
        "status_summary": save_df(status_summary, "status_summary"),
        "state_sales_summary": save_df(state_summary, "state_sales_summary"),
        "city_sales_summary": save_df(city_summary, "city_sales_summary"),
    }

    # Charts
    def line_chart(df_, x, y, title, fname):
        if df_ is None or df_.empty or x not in df_.columns or y not in df_.columns:
            return None
        fig = plt.figure()
        plt.plot(df_[x], df_[y])
        plt.title(title)
        plt.xlabel(x.replace("_"," ").title())
        plt.ylabel(y.replace("_"," ").title())
        plt.xticks(rotation=45, ha="right")
        plt.tight_layout()
        fpath = os.path.join(out_dir, fname)
        plt.savefig(fpath, dpi=150, bbox_inches="tight")
        plt.close(fig)
        return fpath

    def bar_chart(df_, x, y, title, fname, topn=None):
        if df_ is None or df_.empty or x not in df_.columns or y not in df_.columns:
            return None
        data = df_.copy()
        if topn is not None:
            data = data.head(topn)
        fig = plt.figure()
        plt.bar(data[x].astype(str), data[y])
        plt.title(title)
        plt.xlabel(x.replace("_"," ").title())
        plt.ylabel(y.replace("_"," ").title())
        plt.xticks(rotation=45, ha="right")
        plt.tight_layout()
        fpath = os.path.join(out_dir, fname)
        plt.savefig(fpath, dpi=150, bbox_inches="tight")
        plt.close(fig)
        return fpath

    charts = {}
    if timeseries is not None and AMT in timeseries.columns:
        charts["chart_monthly_revenue.png"] = line_chart(timeseries, "year_month", AMT, "Monthly Revenue Trend", "chart_monthly_revenue.png")
    if timeseries is not None and QTY and QTY in timeseries.columns:
        charts["chart_monthly_units.png"] = line_chart(timeseries, "year_month", QTY, "Monthly Units Trend", "chart_monthly_units.png")
    if top_categories_sales is not None:
        charts["chart_top_categories_sales.png"] = bar_chart(top_categories_sales, CAT, AMT, "Top Categories by Sales", "chart_top_categories_sales.png", topn=10)
    if top_categories_units is not None:
        charts["chart_top_categories_units.png"] = bar_chart(top_categories_units, CAT, QTY, "Top Categories by Units", "chart_top_categories_units.png", topn=10)
    if size_summary is not None and AMT in size_summary.columns:
        charts["chart_sales_by_size.png"] = bar_chart(size_summary, SIZE, AMT, "Sales by Size/Variant", "chart_sales_by_size.png", topn=15)
    if fulfil_summary is not None:
        if "orders" in fulfil_summary.columns:
            charts["chart_orders_by_fulfilment.png"] = bar_chart(fulfil_summary.sort_values("orders", ascending=False), FULFIL, "orders", "Orders by Fulfilment Method", "chart_orders_by_fulfilment.png")
        if AMT and AMT in fulfil_summary.columns:
            charts["chart_sales_by_fulfilment.png"] = bar_chart(fulfil_summary.sort_values("sales_amount", ascending=False), FULFIL, "sales_amount", "Sales by Fulfilment Method", "chart_sales_by_fulfilment.png")
    if state_summary is not None and STATE in state_summary.columns:
        charts["chart_top_states_by_sales.png"] = bar_chart(state_summary.head(15), STATE, AMT, "Top States by Sales", "chart_top_states_by_sales.png")
    if city_summary is not None and CITY in city_summary.columns:
        charts["chart_top_cities_by_sales.png"] = bar_chart(city_summary.head(15), CITY, AMT, "Top Cities by Sales", "chart_top_cities_by_sales.png")

    # PowerPoint
    pptx_path = None
    try:
        from pptx import Presentation
        from pptx.util import Inches, Pt
        from pptx.enum.text import MSO_ANCHOR, MSO_AUTO_SIZE

        prs = Presentation()
        slide_layout_title = prs.slide_layouts[0]
        slide_title = prs.slides.add_slide(slide_layout_title)
        slide_title.shapes.title.text = "Amazon Sales Analysis"
        slide_title.placeholders[1].text = f"Generated on {datetime.now():%Y-%m-%d %H:%M}"

        # Use a blank slide layout for KPIs and add a textbox
        slide_layout_blank = prs.slide_layouts[6] # Use a blank layout
        slide_kpis = prs.slides.add_slide(slide_layout_blank)

        # Add title as a textbox
        left, top, width, height = Inches(1), Inches(0.5), Inches(8), Inches(1)
        title_box = slide_kpis.shapes.add_textbox(left, top, width, height)
        title_frame = title_box.text_frame
        title_p = title_frame.add_paragraph()
        title_p.text = "Executive KPIs"
        title_p.font.size = Pt(24) # Adjust font size for title

        left, top, width, height = Inches(1), Inches(1.5), Inches(8), Inches(5)
        txBox = slide_kpis.shapes.add_textbox(left, top, width, height)
        tf = txBox.text_frame
        tf.clear()

        for k,v in OrderedDict(kpi).items():
            p = tf.add_paragraph()
            vfmt = f"{v:,.2f}" if isinstance(v,(int,float)) and not pd.isna(v) else str(v)
            p.text = f"{k.replace('_',' ').title()}: {vfmt}"
            p.font.size = Pt(14) # Adjust font size if needed

        for title, path in charts.items():
            if path and os.path.exists(path):
                slide = prs.slides.add_slide(prs.slide_layouts[5])
                slide.shapes.title.text = title.replace("_"," ").replace(".png","").title()
                slide.shapes.add_picture(path, Inches(0.5), Inches(1.5), height=Inches(5))

        pptx_path = os.path.join(out_dir, "Amazon_Sales_Analysis_Report.pptx")
        prs.save(pptx_path)
    except ImportError:
         print("Python-pptx library not found. Skipping PowerPoint generation.")
         pptx_path = None
    except Exception as e:
        print(f"Error generating PowerPoint: {e}")
        pptx_path = None


    # Text summary
    with open(os.path.join(out_dir, "analysis_summary.txt"), "w", encoding="utf-8") as f:
        f.write("Amazon Sales Analysis – Auto Summary\n")
        f.write(f"Generated: {datetime.now()}\n\n")
        f.write("Detected Columns:\n")
        f.write(", ".join(df.columns) + "\n\n")
        f.write("Key Performance Indicators:\n")
        for k,v in kpi.items():
            vfmt = f"{v:,.2f}" if isinstance(v,(int,float)) and not pd.isna(v) else str(v)
            f.write(f"- {k.replace('_',' ').title()}: {vfmt}\n")

    return {
        "outputs": outputs,
        "charts": charts,
        "pptx_path": pptx_path,
        "kpi": kpi,
        "schema": schema,
        "original_columns": original_columns,
        "standardized_columns": df.columns.tolist(),
    }

# if __name__ == "__main__":
#     import argparse
#     parser = argparse.ArgumentParser()
#     parser.add_argument("--csv", required=True, help="Path to Amazon Sale Report CSV")
#     parser.add_argument("--out", default="analysis_outputs", help="Output directory")
#     args = parser.parse_args()
#     res = main(args.csv, args.out)
#     print("Analysis complete. Outputs:", res["outputs"])
#     print("Charts:", res["charts"])
#     print("PPTX:", res["pptx_path"])

# Call the main function directly with the CSV file path
csv_file_path = "/content/Amazon Sale Report.csv"
output_directory = "analysis_outputs"
res = main(csv_file_path, output_directory)
print("Analysis complete. Outputs:", res["outputs"])
print("Charts:", res["charts"])
print("PPTX:", res["pptx_path"])

/tmp/ipython-input-2834161124.py:45: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df[c] = pd.to_datetime(df[c], errors="coerce", infer_datetime_format=True, dayfirst=False)
/tmp/ipython-input-2834161124.py:45: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df[c] = pd.to_datetime(df[c], errors="coerce", infer_datetime_format=True, dayfirst=False)


Analysis complete. Outputs: {'timeseries': 'analysis_outputs/timeseries_monthly.csv', 'top_categories_sales': 'analysis_outputs/top_categories_by_sales.csv', 'top_categories_units': 'analysis_outputs/top_categories_by_units.csv', 'size_summary': 'analysis_outputs/size_summary.csv', 'fulfilment_summary': 'analysis_outputs/fulfilment_summary.csv', 'status_summary': 'analysis_outputs/status_summary.csv', 'state_sales_summary': 'analysis_outputs/state_sales_summary.csv', 'city_sales_summary': 'analysis_outputs/city_sales_summary.csv'}
Charts: {'chart_monthly_revenue.png': 'analysis_outputs/chart_monthly_revenue.png', 'chart_monthly_units.png': 'analysis_outputs/chart_monthly_units.png', 'chart_top_categories_sales.png': 'analysis_outputs/chart_top_categories_sales.png', 'chart_top_categories_units.png': 'analysis_outputs/chart_top_categories_units.png', 'chart_sales_by_size.png': 'analysis_outputs/chart_sales_by_size.png', 'chart_orders_by_fulfilment.png': 'analysis_outputs/chart_orders_by


# Amazon Sales Analysis Report  

**Student:** Aniruddha Shit
**Project Title:** Amazon Sales Report Analysis  

---

## Key Objectives (from Task Brief)
1. Sales Overview – Understand revenue, orders, units, and trends.  
2. Product/Category Analysis – Identify top categories, variants, sizes.  
3. Fulfilment Analysis – Compare fulfilment methods and order statuses.  
4. Customer/Geographical Segmentation – Revenue contribution by city/state.  
5. Business Insights & Recommendations – Suggest improvements.  

---

## Deliverables

1. Comprehensive Analysis Report – Summarizing findings, insights, and recommendations.  
2. Visualizations – Charts and graphs highlighting trends, categories, fulfilment, and geography.  
3. Insights – Product preferences, customer behaviour, and geographical distribution.  
4. Recommendations – For sales strategy, inventory, and customer service.  

Files and outputs generated:  
- PowerPoint: `Amazon_Sales_Analysis_Report.pptx`  
- Summary Report: `analysis_summary.txt`  
- CSV Files:  
  - `timeseries_monthly.csv`  
  - `top_categories_by_sales.csv`  
  - `top_categories_by_units.csv`  
  - `size_summary.csv`  
  - `fulfilment_summary.csv`  
  - `status_summary.csv`  
  - `state_sales_summary.csv`  
  - `city_sales_summary.csv`  
- Charts: Embedded in PPT and also exported as separate PNG files.  

---

## Comprehensive Analysis

### 1. Sales Overview and Performance Trends
- KPIs Identified: Total revenue, total units sold, total unique orders, and average order value.  
- Monthly Trends: Monthly revenue and unit sales reveal demand peaks and seasonal variations.  
- Insight: Consistent growth in revenue during certain months suggests seasonal promotions or events drive higher sales.  

### 2. Product and Category Analysis
- Top Categories: Identified by revenue and units sold (`top_categories_by_sales.csv`, `top_categories_by_units.csv`).  
- Size/Variant Preferences: Customers showed demand for specific sizes and product variants (`size_summary.csv`).  
- Insight: Some categories contribute disproportionately to revenue, while others sell in volume at low margins.  

### 3. Fulfilment and Status Analysis
- Fulfilment: Orders and revenue compared across fulfilment channels (`fulfilment_summary.csv`).  
- Status: Delivered, Cancelled, and Returned orders analysed (`status_summary.csv`).  
- Insight: Fulfilment by Amazon may dominate sales, but cancellations and returns highlight areas for operational improvement.  

### 4. Customer and Geographical Analysis
- Regional Distribution: Sales mapped by state and city (`state_sales_summary.csv`, `city_sales_summary.csv`).  
- Insight: Revenue is concentrated in a few top-performing states/cities. Regional differences suggest the need for localized strategies.  

---

## Insights on Customer Behaviour
- Product Preferences: High demand for a few top categories and variants.  
- Behavioural Trends: High cancellation and return rates in some categories may be linked to quality or fulfilment issues.  
- Regional Behaviour: Strong markets exist in top 5 cities/states, while weaker regions present opportunities for expansion.  

---

## Recommendations

1. Sales Strategy Improvements  
   - Focus campaigns on top-performing categories.  
   - Re-price or bundle low-margin, high-volume products to increase profitability.  

2. Inventory Management  
   - Align stock levels with geographical demand.  
   - Address high-return categories by improving product information and quality control.  

3. Customer Service Enhancements  
   - Reduce cancellations by improving fulfilment accuracy and delivery times.  
   - Enhance return handling and customer communication to maintain trust.  

---

## Expected Outcome Verification

| Expected Outcome                     | Evidence Source                                                                     | Verified |
|--------------------------------------|-------------------------------------------------------------------------------------|----------|
| Sales trend and KPIs                 | PowerPoint (Executive KPIs, Monthly Revenue Trend), `analysis_summary.txt`          | Yes      |
| Top categories and products           | `top_categories_by_sales.csv`, `top_categories_by_units.csv`, PowerPoint charts     | Yes      |
| Fulfilment analysis                  | `fulfilment_summary.csv`, PowerPoint fulfilment charts                              | Yes      |
| Order status performance             | `status_summary.csv`, PowerPoint chart Orders by Status                             | Yes      |
| State and city contribution          | `state_sales_summary.csv`, `city_sales_summary.csv`, PowerPoint charts              | Yes      |
| Insights and recommendations         | PowerPoint slides, `analysis_summary.txt`, written recommendations in this report   | Yes      |

---

## Additional Recommendations
- RFM Segmentation: If customer IDs are available, segment customers by recency, frequency, and monetary value.  
- Forecasting: Use monthly sales data to project demand and improve planning.  
- Returns Analysis: Combine product category and status data to reduce return rates.  

---

## Conclusion
This project delivers all required outputs: a detailed analysis report, visualization suite, insights into customer behaviour and geography, and actionable recommendations.  

- Comprehensive analysis report explains findings in detail.  
- Visualizations provide clear illustrations of trends and patterns.  
- Insights highlight product preferences, customer behaviour, and geographical distribution.  
- Recommendations propose concrete actions for sales strategies, inventory optimization, and customer service.  

The analysis achieves the expected outcome of providing insights that can optimize operations, improve customer experience, and drive revenue growth.  
